In [53]:
import azure.identity
import os
import openai
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import AzureOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from dotenv import load_dotenv

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

In [54]:
load_dotenv(override=True)

True

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Initialize the Azure OpenAI chat client
chat = ChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    api_key=os.getenv("AZURE_KEY"),
    api_version=os.getenv("AZURE_MODEL"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT")
)

# Example chat input
messages = [
    HumanMessage(content="What's the capital of France?")
]

# Get the response
response = chat.invoke(messages)

# Output the content of the assistant's reply
print(response.content)


c:\Users\markw\repos\Python-GenAI-Course\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3489: UserWarning: WARNING! azure_deployment is not default parameter.
                azure_deployment was transferred to model_kwargs.
                Please confirm that azure_deployment is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\markw\repos\Python-GenAI-Course\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3489: UserWarning: WARNING! api_version is not default parameter.
                api_version was transferred to model_kwargs.
                Please confirm that api_version is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\markw\repos\Python-GenAI-Course\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3489: UserWarning: WARNING! azure_endpoint is not default parameter.
                azure_endpoint was transferred to model_kwargs.
                Please confirm that azure_endpoint is what y

TypeError: Completions.create() got an unexpected keyword argument 'azure_deployment'

In [30]:
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")
AZURE_KEY = os.getenv("AZURE_KEY")

client = ChatCompletionsClient(
    endpoint=AZURE_ENDPOINT,
    credential=AzureKeyCredential(AZURE_KEY)
)

In [31]:
AZURE_MODEL = os.getenv("AZURE_MODEL")

In [32]:
response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="I am going to Milan, what should I see?")
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=AZURE_MODEL
)

print(response.choices[0].message.content)

Milan, located in Italy's northern Lombardy region, is renowned for its fashion, design, and rich cultural heritage. There are numerous attractions to see in Milan, and here's a list of some of the most iconic sights:

1. **Duomo di Milano (Milan Cathedral)**: The city's most famous landmark. This Gothic cathedral took nearly six centuries to complete and is the fifth-largest cathedral in the world. Be sure to visit the rooftop for a breathtaking view of the city.

2. **Galleria Vittorio Emanuele II**: One of the world's oldest shopping malls, boasting a stunning 19th-century glass-vaulted arcade. It's a great place for high-end shopping or simply to enjoy the architecture and perhaps a coffee at one of the elegant cafes.

3. **Santa Maria delle Grazie & The Last Supper**: This church and Dominican convent include the refectory that houses Leonardo da Vinci's "The Last Supper," one of the world's most famous paintings. Remember to book tickets in advance as they sell out quickly.

4. *

In [33]:
response = client.complete(
    model=AZURE_MODEL,
    temperature=0.7,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that makes lots of cat references and uses emojis."},
        {"role": "user", "content": "Write a haiku about a hungry cat who wants tuna"},
    ]
)
print(response.choices[0].message.content)

Whiskers quiver, wait— 🐱
Tuna scent fills the air, mew!
Feast for feline plate. 🍣


In [34]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

text = extract_text_from_pdf('pdf/Wiltshire Item10a_Appendix 1 - MTFS.pdf')

In [35]:
def split_text(text, chunk_size=500, chunk_overlap=50):
    """Split text into smaller chunks."""
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

splits = split_text(text)


In [36]:
def retrieve_relevant_chunks(chunks, query, top_n=3):
    """Retrieve the most relevant chunks based on keyword matching."""
    return sorted(chunks, key=lambda x: query.lower() in x.lower(), reverse=True)[:top_n]


The code cell belows shows  exactly how the function above works.

In [37]:
my_list = ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'abc', 'def', 'ghi', 'jkl', 'mno', 'aaz', 'azz']
test_result = retrieve_relevant_chunks(my_list, "a", top_n=3)
test_result

['aaa', 'abc', 'aaz']

In [38]:
def retrieve_relevant_chunks2(chunks, query, top_n=3):
    """Retrieve the most relevant chunks based on a count of the number of times the query appears in an element of chunks."""
    return sorted(chunks, key=lambda chunk: chunk.lower().count(query.lower()), reverse=True)[:top_n]

In [39]:
# This function works better since it sort on the countr, not simply the existing of the substring in the chunk
test_result = retrieve_relevant_chunks2(my_list, "a", top_n=3)
test_result

['aaa', 'aaz', 'abc']

In [40]:
results = retrieve_relevant_chunks(splits, "finance")
results

['17 November 2022  and outlined a n increase in Department Expenditure Limit for Local \nGovernment of 33% . \nOn the 19 December 2022 the Secretary of State announced the provisional local \ngovernment settlement which provided details on the grant allocations to Councils \nfrom this additional funding, together with other measures aimed at supporting Council \nfinances.   The provisional settlement set out that Local Government Core Spending',
 'SAVINGS  \nThe Cabinet have been working on putting together saving proposals over the three \nyears that would not only still enable business plan priorities to be delivered but also \nfor the Councils finances to be managed and move to a sustainable footing.  \nSaving proposals have been put forward that total £ 51m over the MTFS, significantly \nover £2 6m to be delivered in the first year.  \nThe detail proposal for savings by each service are shown  in annex 7 of this appendix.',
 'Corporate Director People Total 236.926 8.790 - 0.972 3

In [49]:
def create_vector_store(chunks):
    """Convert chunks to embeddings and store in a FAISS vector database."""
    embeddings = OpenAIEmbeddings()

    embeddings = AzureOpenAIEmbeddings(
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        #deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        api_key=os.getenv("AZURE_KEY"),
        api_version=os.getenv("AZURE_MODEL")
    )

    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store

In [42]:
def build_prompt(context, query):
    """ build the RAG prompt from the context and query """
    return f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

print(build_prompt("context......", "questions"))

Context:
context......

Question: questions
Answer:


In [43]:
def generate_response(context, query):
    """Generate a response from the prompt."""
    prompt = build_prompt(context, query)
    response = client.completions.create(model=AZURE_MODEL, prompt=prompt, max_tokens=1000)
    return response.choices[0].text.strip()

In [44]:
pdf_files = ['pdf/' + file for file in os.listdir('pdf')]
pdf_files


['pdf/Cambridge Current-Budget-and-MTFP.pdf',
 'pdf/derbyshire-dales-medium-term-financial-plan-appendix-1.pdf',
 'pdf/middlesborough-council-medium-term-financial-plan.pdf',
 'pdf/North Yorkshire 2023-24-Budget-and-Capital-Programme-and-MTFP-to-2026-27-Police.pdf',
 'pdf/Wiltshire Item10a_Appendix 1 - MTFS.pdf']

In [45]:
# pdf_files = [
#     'pdf/derbyshire-dales-medium-term-financial-plan-appendix-1.pdf', 
#     'pdf/middlesborough-council-medium-term-financial-plan.pdf'
# ]

# Extract text from PDFs and split into chunks
all_chunks = []
for pdf_path in pdf_files:
    text = extract_text_from_pdf(pdf_path)
    chunks = split_text(text)
    all_chunks.extend(chunks)


In [51]:
vector_store = create_vector_store(all_chunks)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

Based on the  query, find the top n matching chunks and combines these into a single text string 

In [50]:
query = "Are the finances of each council on a sound basis?"

search_results = vector_store.similarity_search(query, top_k=10)
contents = [result.page_content for result in search_results]
context = '\n\n'.join(contents)

context


NameError: name 'vector_store' is not defined

In [ ]:
# Use OpenAI to generate a response
response = generate_response(context, query)
print("Response:", response)

